<div class="alert alert-block alert-info"><h1><center> DATA 620_Project3_Document Classification </center></h1></div>


### Author: Juanelle Marks

Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can.

Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the devtest set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set.

How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?

Source: Natural Language Processing with Python, exercise 6.10.2.

### Project Plan

My project will follow the following order:

1. Explore features and classifier used in text NLP Chapter 6. This exploration will give ideas on how i can      incrementally improve the the classifier.
<br>

2. Create features that are guided by outputs from an analysis of the classifiers used in the text.
<br>

3. Evaluate classifiers so as to make inferences about which one may be best.

###  <span style = 'color:red'>List of Headings for Ease of Navigation</span>

Below is a list of main and subheadings used in this project. By project design, each feaurure is an improvement of the other.

* [Load Names Corpus and Libraries](#load_corpus)


* [Part One: Features From Text](#Part_one)

   * [Feature One](#feature_one)

* [Feature Two](#feature_two)

* [Part Two: Improvement Features](#Part_two)

* [Feature Three](#feature_three)

* [Feature Four](#feature_four)

* [Feature Five](#feature_five)

* [Feature Six](#feature_six)

* [Conclusion](#conclusion)

## Load Names Corpus and Libraries<a id = "load_corpus"></a>

Relevant libraries are loaded along with the names corpus. In this section subsetting of the corpus into train,test and devtest is also done.

In [366]:
from nltk.corpus import names
import random
from nltk.classify import apply_features

#### Get Names

In [361]:
%matplotlib inline
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /Users/juanelle/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [362]:
# a  seed is set to make the project reproducible.
random.seed(1234) 
names=([(name, 'male') for name in names.words('male.txt')] +
         [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names) 


In [364]:
print("There are :",len(names), " in this corpus.")

There are : 7944  in this corpus.


In [365]:
#Split data
test_names = names[:500]
devtest_names = names[500:1000]
train_names = names[1000:]

<div class="alert alert-block alert-info"><h2><center>Creating and 
   Evaluating Feature Extractors and Classifiers </center></h2></div>

## Part One: Features From Text <a id = "Part_one"></a><br>

The first two feature extractors used in this project were taken from Chapter 6 of the NLP text. See below an exploration of these extractors.

### Feature 1<a id = "feature_one"></a>

In [402]:
#From text 1: Build feature extractor by looking at final letter in given name
def gender_features(word):
    
    return {'last_letter': word[-1]}


In [403]:
#apply feature to subsets
train_set = [(gender_features(n), g) for (n,g) in train_names]
devtest_set = [(gender_features(n), g) for (n,g) in devtest_names]
test_set = [(gender_features(n), g) for (n,g) in test_names]



In [404]:
#apply classifier to training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [405]:
print(classifier.classify(gender_features('Neo'))) #male
print(classifier.classify(gender_features('Trinity'))) #female

male
female


In [406]:
# Show Accuracy of classifier on all subsets
print("train_set: ", nltk.classify.accuracy(classifier, train_set))
print("test_set: ", nltk.classify.accuracy(classifier, test_set))
print("devtest_set: ", nltk.classify.accuracy(classifier, devtest_set))

train_set:  0.7623847926267281
test_set:  0.776
devtest_set:  0.756


In [407]:
# Show important features - features most effective for distinguising gender by names. 
#This process   guides the design of future feature extractors.
classifier.show_most_informative_features(10)

Most Informative Features
             last_letter = 'k'              male : female =     41.3 : 1.0
             last_letter = 'a'            female : male   =     37.2 : 1.0
             last_letter = 'f'              male : female =     15.4 : 1.0
             last_letter = 'd'              male : female =     10.7 : 1.0
             last_letter = 'm'              male : female =     10.3 : 1.0
             last_letter = 'v'              male : female =      9.9 : 1.0
             last_letter = 'o'              male : female =      8.6 : 1.0
             last_letter = 'p'              male : female =      8.5 : 1.0
             last_letter = 'r'              male : female =      6.8 : 1.0
             last_letter = 'g'              male : female =      5.5 : 1.0


In [408]:
#Check errors
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [409]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Aileen                        
correct=female   guess=male     name=Alisun                        
correct=female   guess=male     name=Alys                          
correct=female   guess=male     name=Arabel                        
correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Bridget                       
correct=female   guess=male     name=Brynn                         
correct=female   guess=male     name=Carol                         
correct=female   guess=male     name=Caroleen                      
correct=female   guess=male     name=Caroljean                     
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Clair                         
correct=female   guess=male     name=Claribel                      
correct=female   guess=male     name=Darb       

In [410]:
print("Error count: ", len(errors))

Error count:  122


This classifier was able to correctly identify the gender of two names given to it.The accuracy of the classifier on the devtest data is approx 76%. This feature extractor and classifier algorithm combination has 122 errors when used on the devtest. We can say that the last letter feature has some effect on the accuracy of the classifier.

###  Feature 2<a id = "feature_two"></a>

In [411]:
#From text 2- An example that shows overfitting
def gender_features1(name):
    features = {}
    features["firstletter"] = name[0].lower()
    features["lastletter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count(%s)" % letter] = name.lower().count(letter)
        features["has(%s)" % letter] = (letter in name.lower())
    return features

In [412]:
#apply feature extractor to subsets
train_set = [(gender_features1(n), g) for (n,g) in train_names]
devtest_set = [(gender_features1(n), g) for (n,g) in devtest_names]
test_set = [(gender_features1(n), g) for (n,g) in test_names]


In [413]:
#apply classifier to training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [415]:
print(classifier.classify(gender_features1('Neo'))) #male
print(classifier.classify(gender_features1('Trinity'))) #female

male
female


In [416]:
# Show Accuracy of classifier on all subsets
print("train_set: ", nltk.classify.accuracy(classifier, train_set))
print("test_set: ", nltk.classify.accuracy(classifier, test_set))
print("devtest_set: ", nltk.classify.accuracy(classifier, devtest_set))

train_set:  0.7805299539170507
test_set:  0.762
devtest_set:  0.758


In [417]:
# Show important features - features most effective for distinguising gender by names. 
#This process   guides the design of future feature extractors.
classifier.show_most_informative_features(20)

Most Informative Features
              lastletter = 'k'              male : female =     41.3 : 1.0
              lastletter = 'a'            female : male   =     37.2 : 1.0
              lastletter = 'f'              male : female =     15.4 : 1.0
              lastletter = 'd'              male : female =     10.7 : 1.0
              lastletter = 'm'              male : female =     10.3 : 1.0
              lastletter = 'v'              male : female =      9.9 : 1.0
                count(v) = 2              female : male   =      8.8 : 1.0
              lastletter = 'o'              male : female =      8.6 : 1.0
              lastletter = 'p'              male : female =      8.5 : 1.0
              lastletter = 'r'              male : female =      6.8 : 1.0
              lastletter = 'g'              male : female =      5.5 : 1.0
              lastletter = 'z'              male : female =      5.1 : 1.0
                count(a) = 3              female : male   =      5.0 : 1.0

In [418]:
#Check errors
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features1(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [419]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Ardyth                        
correct=female   guess=male     name=Audry                         
correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Bridget                       
correct=female   guess=male     name=Brooke                        
correct=female   guess=male     name=Brynn                         
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Chery                         
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Corry                         
correct=female   guess=male     name=Darb                          
correct=female   guess=male     name=Dorolice                      
correct=female   guess=male     name=Fawne                         
correct=female   guess=male     name=Gennifer                      
correct=female   guess=male     name=Gretchen   

In [420]:
print("Error count: ", len(errors))

Error count:  121


This classifier was also able to correctly identify the gender of two names given to it.The accuracy of the classifier on the devtest data the same as that in feature one. This feature extractor and classifier algorithm combination however had one less error. We can say that the last letter feature has some effect on the accuracy of the classifier.This  feature extractor and classifier combination has 121 errors when used on the devtest. Though fitted with more features, this feature extractor did not significantly reduce errors. Based on classifier accuracy and most_informative_features, feature one nd feature do not vary much in what they can achieve with respect to improving accuracy of gender classification.<br>

## Part Two:  Improvement  Features<a id = "Part_Two"></a>

### Feature 3<a id = "feature_three"></a>

In [421]:
#Based on the informative features list, it is evident that  male and female names 
#differ a bit according letters found at the beginning and ending of their names. It is on this premise that
#improvements will be made.

#To improve on  feature extractor2 above, number 3 will include 2 letter suffix and 2 letter prefix to list of features
def gender_features3(name):
    features = {}
    features["firstletter"] = name[0].lower()
    features["lastletter"] = name[-1].lower()
    features["suffix2"]= name[-2:].lower()
    features["prefix2"]= name[:2].lower()
    for letter in 'aeiou':
        features["count(%s)" % letter] = name.lower().count(letter)
        features["has(%s)" % letter] = (letter in name.lower())
    return features

In [422]:
#apply feature extractor to subsets
train_set = [(gender_features3(n), g) for (n,g) in train_names]
devtest_set = [(gender_features3(n), g) for (n,g) in devtest_names]
test_set = [(gender_features3(n), g) for (n,g) in test_names]


In [423]:
#apply classifier to training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [424]:
print(classifier.classify(gender_features3('Neo'))) #male
print(classifier.classify(gender_features3('Trinity'))) #female

male
female


In [425]:
# Show Accuracy of classifier on all subsets
print("train_set: ", nltk.classify.accuracy(classifier, train_set))
print("test_set: ", nltk.classify.accuracy(classifier, test_set))
print("devtest_set: ", nltk.classify.accuracy(classifier, devtest_set))

train_set:  0.8086117511520737
test_set:  0.804
devtest_set:  0.784


This improvement in accuracy is somewhat significant. The accuracy of of classifying the devtest subset increased by approximately 3%

In [426]:
# Show important features - features most effective for distinguising gender by names. 
#This process   guides the design of future feature extractors.
classifier.show_most_informative_features(20)

Most Informative Features
                 suffix2 = 'na'           female : male   =    157.4 : 1.0
                 suffix2 = 'la'           female : male   =     69.8 : 1.0
                 suffix2 = 'ta'           female : male   =     43.3 : 1.0
              lastletter = 'k'              male : female =     41.3 : 1.0
                 suffix2 = 'ld'             male : female =     39.7 : 1.0
                 suffix2 = 'us'             male : female =     39.2 : 1.0
              lastletter = 'a'            female : male   =     37.2 : 1.0
                 suffix2 = 'ia'           female : male   =     37.1 : 1.0
                 suffix2 = 'sa'           female : male   =     32.4 : 1.0
                 suffix2 = 'rt'             male : female =     29.8 : 1.0
                 suffix2 = 'rd'             male : female =     28.5 : 1.0
                 suffix2 = 'do'             male : female =     26.3 : 1.0
                 suffix2 = 'ra'           female : male   =     24.0 : 1.0

By including 2 letter suffix in the extractor it was greatly improved.

In [427]:
#Check errors
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features3(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [428]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Audry                         
correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Bridget                       
correct=female   guess=male     name=Brooke                        
correct=female   guess=male     name=Brynn                         
correct=female   guess=male     name=Carol                         
correct=female   guess=male     name=Chery                         
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Clair                         
correct=female   guess=male     name=Corry                         
correct=female   guess=male     name=Darb                          
correct=female   guess=male     name=Dorolice                      
correct=female   guess=male     name=Gennifer                      
correct=female   guess=male     name=Gillan                        
correct=female   guess=male     name=Gretchen   

In [429]:
print("Error count: ", len(errors))

Error count:  108


This classifier was able also to correctly identify the gender of two random names given to it.The accuracy of the classifier on the devtest data improved to approx 78%. This feature extractor and classifier algorithm combination has 108 errors when used on the devtest.This is indeed great improvement. We can conclude that including first and last  letters in  features somewhat improves the accuracy of the classifier.There is also a significant reduction in errors. Thus feature 3 along with the naive bayes classifier is the best combination thus far.

### Feature 4<a id = "feature_four"></a>

In [430]:
## We will improve feature extractor by including multiple fits and last letters 
#as well as checking the vowel the name ends with 
def gender_features4(word):
    return {'first3_letter': word[0:3],
            'last2_letter': word[-2:],
            'last_is_vowel' : (word[-1] in 'aeiouy')}

In [431]:
#apply feature extractor to subsets
train_set = [(gender_features4(n), g) for (n,g) in train_names]
devtest_set = [(gender_features4(n), g) for (n,g) in devtest_names]
test_set = [(gender_features4(n), g) for (n,g) in test_names]


In [432]:
#apply classifier to training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [433]:
print(classifier.classify(gender_features4('Neo'))) #male
print(classifier.classify(gender_features4('Trinity'))) #female

male
female


In [434]:
# Show Accuracy of classifier on all subsets
print("train_set: ", nltk.classify.accuracy(classifier, train_set))
print("test_set: ", nltk.classify.accuracy(classifier, test_set))
print("devtest_set: ", nltk.classify.accuracy(classifier, devtest_set))

train_set:  0.8542626728110599
test_set:  0.804
devtest_set:  0.808


In [435]:
# Show important features - features most effective for distinguising gender by names. 
#This process   guides the design of future feature extractors.
classifier.show_most_informative_features(20)

Most Informative Features
                 suffix2 = 'na'           female : male   =    157.5 : 1.0
                 suffix2 = 'la'           female : male   =     69.5 : 1.0
                 suffix2 = 'ta'           female : male   =     43.4 : 1.0
                 suffix2 = 'ld'             male : female =     39.6 : 1.0
                 suffix2 = 'us'             male : female =     39.2 : 1.0
                 suffix2 = 'ia'           female : male   =     37.1 : 1.0
                 suffix2 = 'sa'           female : male   =     32.5 : 1.0
                 suffix2 = 'rt'             male : female =     29.8 : 1.0
                 suffix2 = 'rd'             male : female =     28.5 : 1.0
                 suffix2 = 'do'             male : female =     26.2 : 1.0
                 suffix2 = 'ra'           female : male   =     24.0 : 1.0
                 suffix2 = 'ch'             male : female =     22.9 : 1.0
           first3_letter = 'Dor'          female : male   =     16.4 : 1.0

In [436]:
#Check errors
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features4(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [437]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Arabel                        
correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Brooke                        
correct=female   guess=male     name=Brynn                         
correct=female   guess=male     name=Carol                         
correct=female   guess=male     name=Caroljean                     
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Clair                         
correct=female   guess=male     name=Claribel                      
correct=female   guess=male     name=Darb                          
correct=female   guess=male     name=Dierdre                       
correct=female   guess=male     name=Elizabeth                     
correct=female   guess=male     name=Gennifer                      
correct=female   guess=male     name=Gillan     

In [438]:
print("Error count: ", len(errors))

Error count:  96


This classifier was able also to correctly identify the gender of two random names given to it.The accuracy of the classifier on the devtest data improved to approx 81%. This is an even better classifier! This feature extractor and classifier algorithm combination has 96 errors when used on the devtest.This is indeed great improvement. We can conclude that including multiple first and last  letters letters along with checking the ending vowel in  features somewhat improves the accuracy of the classifier.Thus feature 4 along with the naive bayes classifier is the best combination thus far.Errors reduced to 96 significantly improved accuracy.

### Feature 5<a id = "feature_five"></a>

In [440]:
## this feature extends the number of last letters
def gender_features5(word):
    return {'first3_letter': word[0:3],
            'last3_letter': word[-3:],
            'last_is_vowel' : (word[-1] in 'aeiouy')}

In [441]:
#apply feature extractor to subsets
train_set = [(gender_features5(n), g) for (n,g) in train_names]
devtest_set = [(gender_features5(n), g) for (n,g) in devtest_names]
test_set = [(gender_features5(n), g) for (n,g) in test_names]


In [442]:
#apply classifier to training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [336]:
print(classifier.classify(gender_features5('Neo'))) #male
print(classifier.classify(gender_features5('Trinity'))) #female

female
female


In [443]:
# Show Accuracy of classifier on all subsets
print("train_set: ", nltk.classify.accuracy(classifier, train_set))
print("test_set: ", nltk.classify.accuracy(classifier, test_set))
print("devtest_set: ", nltk.classify.accuracy(classifier, devtest_set))

train_set:  0.8832085253456221
test_set:  0.828
devtest_set:  0.818


In [444]:
# Show important features - features most effective for distinguising gender by names. 
#This process   guides the design of future feature extractors.
classifier.show_most_informative_features(20)

Most Informative Features
            last3_letter = 'ita'          female : male   =     25.9 : 1.0
            last3_letter = 'ana'          female : male   =     24.6 : 1.0
            last3_letter = 'ard'            male : female =     19.8 : 1.0
            last3_letter = 'nne'          female : male   =     18.6 : 1.0
           first3_letter = 'Dor'          female : male   =     16.4 : 1.0
           first3_letter = 'Mel'          female : male   =     15.9 : 1.0
            last3_letter = 'ert'            male : female =     15.2 : 1.0
           first3_letter = 'Tha'            male : female =     14.1 : 1.0
            last3_letter = 'ria'          female : male   =     13.5 : 1.0
            last3_letter = 'ela'          female : male   =     13.1 : 1.0
            last3_letter = 'vin'            male : female =     13.0 : 1.0
           first3_letter = 'Cat'          female : male   =     12.1 : 1.0
            last3_letter = 'mon'            male : female =     11.9 : 1.0

In [445]:
#Check errors
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features5(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [446]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Brooke                        
correct=female   guess=male     name=Brynn                         
correct=female   guess=male     name=Carol                         
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Clair                         
correct=female   guess=male     name=Cookie                        
correct=female   guess=male     name=Darb                          
correct=female   guess=male     name=Emili                         
correct=female   guess=male     name=Gillan                        
correct=female   guess=male     name=Gretchen                      
correct=female   guess=male     name=Heath                         
correct=female   guess=male     name=Ingaberg                      
correct=female   guess=male     name=Ingrid     

In [447]:
print("Error count: ", len(errors))

Error count:  91


This classifier incorrectly identifies the gender of two random names given to it.The accuracy of the classifier on the devtest data improved to approx 82%. This feature extractor and classifier algorithm combination has 91 errors when used on the devtest.This is an improvement over the precious feature. We can conclude that including first and last  letters in  features somewhat improves the accuracy of the classifier.There is also a significant reduction in errors. Thus feature 5 along with the naive bayes classifier is the best combination thus far.

### Feature 6<a id = "feature_six"></a>

In [448]:
##looks take both 3rd and 2nd last letters.
def gender_features6(word):
    return {'first3_letter': word[0:3],
            'last3_letter': word[-3:],
            'last2_letter': word[-3:],
            'last_is_vowel' : (word[-1] in 'aeiouy')}

In [449]:
#apply feature extractor to subsets
train_set = [(gender_features6(n), g) for (n,g) in train_names]
devtest_set = [(gender_features6(n), g) for (n,g) in devtest_names]
test_set = [(gender_features6(n), g) for (n,g) in test_names]


In [450]:
#apply classifier to training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [451]:
print(classifier.classify(gender_features6('Neo'))) #male
print(classifier.classify(gender_features6('Trinity'))) #female

male
female


In [452]:
# Show Accuracy of classifier on all subsets
print("train_set: ", nltk.classify.accuracy(classifier, train_set))
print("test_set: ", nltk.classify.accuracy(classifier, test_set))
print("devtest_set: ", nltk.classify.accuracy(classifier, devtest_set))

train_set:  0.8847926267281107
test_set:  0.822
devtest_set:  0.834


In [453]:
# Show important features - features most effective for distinguising gender by names. 
#This process   guides the design of future feature extractors.
classifier.show_most_informative_features(20)

Most Informative Features
            last2_letter = 'ita'          female : male   =     25.9 : 1.0
            last3_letter = 'ita'          female : male   =     25.9 : 1.0
            last3_letter = 'ana'          female : male   =     24.6 : 1.0
            last2_letter = 'ana'          female : male   =     24.6 : 1.0
            last3_letter = 'ard'            male : female =     19.8 : 1.0
            last2_letter = 'ard'            male : female =     19.8 : 1.0
            last3_letter = 'nne'          female : male   =     18.6 : 1.0
            last2_letter = 'nne'          female : male   =     18.6 : 1.0
           first3_letter = 'Dor'          female : male   =     16.4 : 1.0
           first3_letter = 'Mel'          female : male   =     15.9 : 1.0
            last2_letter = 'ert'            male : female =     15.2 : 1.0
            last3_letter = 'ert'            male : female =     15.2 : 1.0
           first3_letter = 'Tha'            male : female =     14.1 : 1.0

In [454]:
#Check errors
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features6(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [455]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Brooke                        
correct=female   guess=male     name=Calley                        
correct=female   guess=male     name=Carol                         
correct=female   guess=male     name=Charis                        
correct=female   guess=male     name=Chery                         
correct=female   guess=male     name=Christin                      
correct=female   guess=male     name=Clair                         
correct=female   guess=male     name=Cookie                        
correct=female   guess=male     name=Edna                          
correct=female   guess=male     name=Emili                         
correct=female   guess=male     name=Gillan                        
correct=female   guess=male     name=Gretchen                      
correct=female   guess=male     name=Halley                        
correct=female   guess=male     name=Heath      

In [456]:
print("Error count: ", len(errors))

Error count:  83


This classifier was able also to correctly identify the gender of two random names given to it.The accuracy of the classifier on the devtest data improved to approx 83%. This feature extractor and classifier algorithm combination has only 83 errors when used on the devtest.This is indeed great improvement. We can conclude that including multiple first and last  letters in  features somewhat improves the accuracy of the classifier.There is also a significant reduction in errors. Thus feature 6 along with the naive bayes classifier is the best combination thus far.

## Conclusion<a id = "conclusion"></a>

By checking for multiple first and last letters as well as if last letter is a vowel, one can incrementally improve the gender classification of names. There is still room for improving the accuracy of the classifier. One way this  could be done is by using the feature extractor with a different classifying algorithm such as  a decision tree. Throughout the project, the classifier's performance across three subsets of data was continually observed. There was no significant difference between the two across each trial. The accuracy of the classifier on each subset both showed incremenal improvement.